# 0.0. IMPORTS

In [1]:
import math
import numpy  as np
import pandas as pd
import random
import pickle
import warnings
import inflection
import seaborn as sns
import xgboost as xgb
import matplotlib

from scipy.stats           import kurtosis
from scipy                 import stats  as ss
from boruta                import BorutaPy
from matplotlib            import pyplot as plt
from IPython.display       import Image
from IPython.core.display  import HTML



## 0.1. Helper functions

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [3]:
jupyter_settings();

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


C:\Users\Emanoel\anaconda3\envs\analise_de_dados\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


## 0.2. Loading data

In [4]:
df_train_raw = pd.read_csv('../data/train.csv' , low_memory=False)

#df_test_raw = pd.read_csv('../data/test.csv' , low_memory=False)

In [5]:
df_train_raw.sample()

,date,store,item,sales
889481,2013-08-08,8,49,29


# 1.0. DATA DESCRIPTION

In [6]:
df1 = df_train_raw.copy()

In [7]:
df1.columns

Index(['date', 'store', 'item', 'sales'], dtype='object')

## 1.2. Data dimensions

In [8]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of columns: {}'.format(df1.shape[1]))

Number of rows: 913000
Number of columns: 4


## 1.3. Data types

In [9]:
df1.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [10]:
df1.dtypes

date     object
store     int64
item      int64
sales     int64
dtype: object

In [11]:
# transformation date
df1['date'] = pd.to_datetime(df1['date'])

# transformation sales
df1['sales' ] = df1['sales' ].astype('float64')

In [12]:
df1.dtypes

date     datetime64[ns]
store             int64
item              int64
sales           float64
dtype: object

## 1.4. Check NA

In [13]:
df1.isna().sum()

date     0
store    0
item     0
sales    0
dtype: int64

## 1.5. Descriptive statistics

In [14]:
# separação variáveis numéricas e categoricas

num_attributes = df1.select_dtypes(include = ['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude = ['int64', 'float64', 'datetime64[ns]'])

In [15]:
# medidas de tendência central - mean and median

ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T


# medidas de dispersão - std, min, max, range, skew, kurtosis

md1 = pd.DataFrame( num_attributes.apply( np.std ) ).T 
md2 = pd.DataFrame( num_attributes.apply( min ) ).T 
md3 = pd.DataFrame( num_attributes.apply( max ) ).T 
md4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T 
md5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T 
md6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T 

#concatenar

medidas = pd.concat([ct1, ct2, md2, md3, md1, md4, md5, md6 ]).T.reset_index()
medidas.columns = ['attributes', 'mean', 'median', 'min', 'max', 'std', 'range', 'skew', 'kurtosis' ]

In [16]:
medidas

,attributes,mean,median,min,max,std,range,skew,kurtosis
0,store,5.500000,5.5,1.0,10.0,2.872281,9.0,0.000000,-1.224243
1,item,25.500000,25.5,1.0,50.0,14.430870,49.0,0.000000,-1.200960
2,sales,52.250287,47.0,0.0,231.0,28.801128,231.0,0.867112,0.509070


In [17]:
df1.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [ ]:
sns.displot( data = df1['sales'], kde=True );

In [ ]:
sns.boxplot( data = df1['sales'], orient="h", color='skyblue' );

In [ ]:
num_attributes.apply( lambda x : x.unique().shape[0])

In [ ]:
aux = df1[(df1['store'] != '0') & (df1['sales'] > 0)]

plt.subplot();
sns.boxplot( x='store', y='sales', data=aux );

# 2.0. FEATURE ENGINEERING